In [ ]:
import scispacy
import spacy
import pandas as pd
import sys
from datetime import datetime 

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import json
from itertools import combinations
from scipy import stats
from collections import Counter
import multiprocessing, time

In [4]:
nlp = spacy.load("en_core_sci_scibert")

/users/k1810895/.conda/envs/py39/lib/python3.9/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [5]:
rels = json.load(open('data/rel_map.json', 'r'))

In [6]:
apath = '/users/k1810895/data/KER/data/scispacy_sentences/'
bpath = '/users/k1810895/data/KER/data/medcat_sentence/'

In [7]:
notes = pd.read_csv('/users/k1810895/data/KER/data/mimic_notes.csv')

/tmp/ipykernel_141079/134885324.py:1: DtypeWarning: Columns (4,5,15) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv('/users/k1810895/data/KER/data/mimic_notes.csv')


In [8]:
id_text = pd.Series(notes.text.values,index=notes.row_id).to_dict()

In [9]:
def screen_docs(path, output_file):
    with open(output_file, 'w', buffering=128) as fw:
        for index in range(1000):
#             path = apath
#             index = 0
            file_name = 'batch' + str(index) + '.json'

            docs = json.load(open(path+file_name, 'r'))

            for did in docs.keys():
            #   Only for discharge summary since the quality of the content is relatively higher
            #   if int(did) in discharg_dids:
                if True:
                    entities = []
                    tokens = []
                    text = id_text[int(did)]
                    doc = nlp(text)
            #         print(text)
                    for s in doc.sents:
                        for token in s:
                            tokens.append(token.text)

                    sents = docs[did]
                    for sid in sents.keys():
                        sent_start = int(sid.split('_')[0])
                        sent = sents[sid]

                        annots = sent['sent_annots']            
                        for i, ann in enumerate(annots):
                            ann['id'] = len(entities)
                            ann['start_char'] = ann['start_char'] + sent_start
                            ann['end_char'] = ann['end_char'] + sent_start
                            span = doc.char_span(ann['start_char'], ann['end_char'])
                            ann['start'] = span.start
                            ann['end'] = span.end
            #                 print(ann['text'], '-----', tokens[ann['start']: ann['end']], '----', text[ann['start_char']: ann['end_char']])
                            entities.append(ann)
                    relations = []
                    # If include all sentence with any number of annotations. it ends with a super large file
                    # Select centain types of annotations to focus. 
                    for index_ai, ai in enumerate(entities):
                        for index_aj, aj in enumerate(entities[index_ai+1:]):
                            index_aj = index_aj + index_ai + 1
                            ai_cui, aj_cui = ai['cui'], aj['cui']

                            min_cui, max_cui = ai_cui, aj_cui
                            min_cui_index, max_cui_index = index_ai, index_aj

                            if min_cui > max_cui:
                                min_cui, max_cui = aj_cui, ai_cui
                                min_cui_index, max_cui_index = index_aj, index_ai

                            cui_key = min_cui + '-' + max_cui                    
                            if cui_key in rels:

                                rel = rels[cui_key]['rela']
                                dire = rels[cui_key]['dir']
                                if dire == 'Y':
                                    head, tail = min_cui_index, max_cui_index

                                else:
                                    head, tail = max_cui_index, min_cui_index
            #                         print(cui_key, rels[cui_key])
            #                     print({'type': rel,  'head': (entities[head]['cui']), 'tail': entities[tail]['cui']})
                                relations.append({'type': rel,  'head': head, 'tail': tail})
                    fw.write(json.dumps({'text': text, 'tokens': tokens, 'entities': entities, 'relations': relations}))
                    fw.write('\n')


In [20]:
screen_docs(apath, '/users/k1810895/data/KER/data/scispacy_docs_v2.txt')

KeyboardInterrupt: 

In [18]:
# multiprocessing.Process(target=screen_docs, args=(apath, '/users/k1810895/data/KER/data/scispacy_docs.txt')).start()

In [17]:
# !top